# Evaluation Result

In [1]:
from sklearn import metrics
from pylib.tlsh_lib import *
import numpy as np
import pandas as pd
import time
tic = time.perf_counter() # experiment time counter 

In [2]:
#Remove NaN function
nan = False
def containNan(labelList):
    for lable in labelList:
        if lable == 'n/a':
            nan = True
            return nan
        
def removeNan(hashlist, labelList):
    count = -1
    newhashlist = []
    newlabelList = []
    
    for lable in labelList:
        count += 1
        if lable != 'n/a':
            newhashlist.append(hashlist[count])
            newlabelList.append(labelList[count])
    return newhashlist, newlabelList

In [3]:
datafile = "dataDir2/mb_1K.csv" #<----------Change this file size

(path,file) = datafile.split("/") #save file path
(filename,filetype) = file.split(".") #save file type

(tlist, [labelList, dateList, hashList]) = tlsh_csvfile(datafile) # return (tlist, [labelList, dateList, hashList])
#(tlist, labelList) = tlsh_csvfile(datafile) 

#remove Nan Value
#(tlist, labelList) = removeNan(tlist, labelList)

print("Number of samples is " + str(len(tlist)))
print("Number of Unique Lable is " + str(len(set(labelList))))
print(hashList[0:10])
nlable = len(set(labelList))
nan = containNan(labelList)

Number of samples is 1000
Number of Unique Lable is 56
['12288:TcinWHCM2K4C2fut2I2U9WLDQ9RnX/lmSG02h8W62NtYy:TR3CBtNTpWNT6y', '12288:3b1f+WVBULxM6IdO6r1uH38W9D+4ncY2BdcvwGqIg3/FTj32c4zWKO:rB+s8eOOI38z4cXSwKg3p7Vx', '12288:lH1rtcf9uFvwrp+8uGqARQi1+3lwbcSpsPP7r9r/+ppppppppppppppppppppppJ:VzF8oQXSi1Oevw1q', '12288:0jdRStsZZzk8/dRStsZZzkldRStsZZzkbgoPYj3YnSn93kEcnVAcx6lQr1Mjg9W2:NCdktCdkiCdkbkjInJmcIlQGjZCRClk', '12288:sULkUtcX1J7ygKXcNxa64qnYDNWXpcJyQPRI8rv:sUL9tcFJ7ygAcUWsyiRI87', '768:dLt2gJbT/5wQu2rNnFfIr8t7sJmn3P5DBlyF8yPyeJ8YH+Rs+FWYQw+6VzBTPv7r:L3JbTBwQuA5Fm86+dBlyFzyeJ8U+FWYj', '12288:dYWWHCM2K4CUY4lbFe5h5LtEjthcgbdQtAa+xaan:J3CUH9KLOTFtxa+', '12288:vjBID6xsmQECVwPvjY/P5vaDx9r4A7Pm6r0c8Ayfk4MwzqEj:a5nGahKx9lrVr0nAT10j', '12288:9ULkUtcX1J7ygKXcNxa64qnYDNWXpcJyQPRI8rA:9UL9tcFJ7ygAcUWsyiRI8s', '24576:G+oELLuVzvLM223WfM8oFJYEVOdsCDd3lF4iDWz3lbWaDTvT:hoEfkzDM2sL862tDd3AQWz1blPr']


In [4]:
df = pd.DataFrame()

def getResult(fh,c,labelList, clusterNumber):
    d = {word: key for key, word in enumerate(set(labelList))}
    labelList_id = [d[word] for word in labelList]
    a = np.array(labelList_id).reshape(-1, 1)
    
    #decimal place
    dp = 4
    
    homo = round(metrics.homogeneity_score(labelList_id, clusterNumber),dp)
    silh = round(metrics.silhouette_score(a, clusterNumber, metric='euclidean'),dp)
    cali = round(metrics.calinski_harabasz_score(a, clusterNumber),dp)
    dav = round(metrics.davies_bouldin_score(a, clusterNumber),dp)
    
    print("Homogeneity score is " + str(homo))
    print("Silhouette score is " + str(silh))
    print("Calinski harabasz score is " + str(cali))
    print("Davies bouldin score is " + str(dav))
    
    result = {"File": str(filename),
              "nSample": int(len(tlist)),
              "Hash": str(fh),
              "Cluster": str(c),
              "Has_n/a": bool(nan),
              "nLabel": int(nlable),
              "nCluster": int(max(clusterNumber)),
              "Time(s)": float(end),
              "Homo.":float(homo),
              "Sil.":float(silh),
              "Cal.":float(cali),
              "Dav.":float(dav)}
    return result

# TLSH Agglomerative Clustering

In [5]:
start = time.perf_counter()

clusterNumber = assignCluster(tlist, nlable)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_hac_out.txt"
#outputClusters(outfile, tlist, clusterNumber, labelList, quiet=True)

dict = getResult("tlsh","hac",labelList,clusterNumber)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.5555 seconds
Homogeneity score is 0.4868
Silhouette score is -0.5577
Calinski harabasz score is 5.7499
Davies bouldin score is 143.9414


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.5555,0.4868,-0.5577,5.7499,143.9414


# TLSH HAC-T

In [6]:
from pylib.hac_lib  import *

hac_resetDistCalc()
start = time.perf_counter()

res = HAC_T(datafile, CDist=30, step3=0, outfname="tmp.txt", cenfname="tmp2.txt") 

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_hac-t_out.txt"
#outputClusters(outfile, tlist, res, labelList, quiet=True)

nclusters = max(res)
nDistCalc = hac_lookupDistCalc()

print("Number of cluster is " + str(nclusters))
print("Number of Distance Calculated is " + str(nDistCalc))

dict = getResult("tlsh","hac-t",labelList,res)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.235 seconds
Number of cluster is 85
Number of Distance Calculated is 227245
Homogeneity score is 0.3453
Silhouette score is -0.7047
Calinski harabasz score is 3.4985
Davies bouldin score is 35.8058


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.5555,0.4868,-0.5577,5.7499,143.9414
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2350,0.3453,-0.7047,3.4985,35.8058


# TLSH DBSCAN

In [7]:
resetDistCalc()
start = time.perf_counter()

res = runDBSCAN(tlist, eps=30, min_samples=2, algorithm='auto')

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_dbscan_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList, quiet=True)

nclusters = max(res.labels_)
nDistCalc = lookupDistCalc()

print("nclusters is " + str(nclusters))
print("nDistCalc is " + str(nDistCalc))

dict = getResult("tlsh","dbscan",labelList,res.labels_)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.9082 seconds
nclusters is 71
nDistCalc is 961443
Homogeneity score is 0.3336
Silhouette score is -0.7083
Calinski harabasz score is 4.1939
Davies bouldin score is 24.7111


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.5555,0.4868,-0.5577,5.7499,143.9414
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2350,0.3453,-0.7047,3.4985,35.8058
2,mb_1K,1000.0,tlsh,dbscan,1.0,56.0,71.0,0.9082,0.3336,-0.7083,4.1939,24.7111


# TLSH KMeans

In [8]:
from sklearn.cluster import KMeans

def runKMean(tlist, n):
    tdata = tlist2cdata(tlist)
    res = KMeans(n_clusters=n, random_state=0).fit(tdata)
    return(res)

In [9]:
start = time.perf_counter()

res = runKMean(tlist, nlable)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_kmean_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList)

dict = getResult("tlsh","kmeans",labelList,res.labels_)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.1688 seconds
Homogeneity score is 0.4641
Silhouette score is -0.6218
Calinski harabasz score is 3.9487
Davies bouldin score is 342.2735


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.5555,0.4868,-0.5577,5.7499,143.9414
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2350,0.3453,-0.7047,3.4985,35.8058
2,mb_1K,1000.0,tlsh,dbscan,1.0,56.0,71.0,0.9082,0.3336,-0.7083,4.1939,24.7111
3,mb_1K,1000.0,tlsh,kmeans,1.0,56.0,55.0,0.1688,0.4641,-0.6218,3.9487,342.2735


In [10]:
res.labels_

array([28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 34, 34, 34, 34, 34, 34, 34, 34,
       34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 38, 38, 38, 38, 38, 38, 38,
       38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 46,
       46, 46, 46, 46, 46

### Output

In [11]:
outfile = path + "/output/" + filename + "_result.csv"
df.to_csv(outfile, index = False)

In [12]:
toc = round(time.perf_counter() - tic,4)
toc

2.2239

# Affinity Propagation

In [13]:
from sklearn.cluster import AffinityPropagation

def runAP(tlist,n):
    tdata = tlist2cdata(tlist)
    #print(tlist)
    print(tdata)
    res = AffinityPropagation(random_state=5).fit(tdata)
    return(res)

In [14]:
start = time.perf_counter()

res = runAP(tlist,5)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_kmean_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList)

#dict = getResult("tlsh","ap",labelList,res.labels_)
#df = df.append(dict, ignore_index = True)
#df

[[  0]
 [  1]
 [  2]
 [  3]
 [  4]
 [  5]
 [  6]
 [  7]
 [  8]
 [  9]
 [ 10]
 [ 11]
 [ 12]
 [ 13]
 [ 14]
 [ 15]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 20]
 [ 21]
 [ 22]
 [ 23]
 [ 24]
 [ 25]
 [ 26]
 [ 27]
 [ 28]
 [ 29]
 [ 30]
 [ 31]
 [ 32]
 [ 33]
 [ 34]
 [ 35]
 [ 36]
 [ 37]
 [ 38]
 [ 39]
 [ 40]
 [ 41]
 [ 42]
 [ 43]
 [ 44]
 [ 45]
 [ 46]
 [ 47]
 [ 48]
 [ 49]
 [ 50]
 [ 51]
 [ 52]
 [ 53]
 [ 54]
 [ 55]
 [ 56]
 [ 57]
 [ 58]
 [ 59]
 [ 60]
 [ 61]
 [ 62]
 [ 63]
 [ 64]
 [ 65]
 [ 66]
 [ 67]
 [ 68]
 [ 69]
 [ 70]
 [ 71]
 [ 72]
 [ 73]
 [ 74]
 [ 75]
 [ 76]
 [ 77]
 [ 78]
 [ 79]
 [ 80]
 [ 81]
 [ 82]
 [ 83]
 [ 84]
 [ 85]
 [ 86]
 [ 87]
 [ 88]
 [ 89]
 [ 90]
 [ 91]
 [ 92]
 [ 93]
 [ 94]
 [ 95]
 [ 96]
 [ 97]
 [ 98]
 [ 99]
 [100]
 [101]
 [102]
 [103]
 [104]
 [105]
 [106]
 [107]
 [108]
 [109]
 [110]
 [111]
 [112]
 [113]
 [114]
 [115]
 [116]
 [117]
 [118]
 [119]
 [120]
 [121]
 [122]
 [123]
 [124]
 [125]
 [126]
 [127]
 [128]
 [129]
 [130]
 [131]
 [132]
 [133]
 [134]
 [135]
 [136]
 [137]
 [138]
 [139]
 [140]
 [141]
 [142]

C:\Users\user\anaconda3\envs\tlshCluster\lib\site-packages\sklearn\cluster\_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(


In [15]:
res

AffinityPropagation(random_state=5)

In [16]:
res.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

# Mean Shift

In [17]:
from sklearn.cluster import MeanShift

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = MeanShift(bandwidth=2).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

Code ran in 0.535 seconds


MeanShift(bandwidth=2)

In [18]:
res.labels_

array([332, 332, 332, 331, 331, 331, 330, 330, 330, 329, 329, 329, 328,
       328, 328, 327, 327, 327, 326, 326, 326, 325, 325, 325, 324, 324,
       324, 323, 323, 323, 322, 322, 322, 321, 321, 321, 320, 320, 320,
       319, 319, 319, 318, 318, 318, 317, 317, 317, 316, 316, 316, 315,
       315, 315, 314, 314, 314, 313, 313, 313, 312, 312, 312, 311, 311,
       311, 310, 310, 310, 309, 309, 309, 308, 308, 308, 307, 307, 307,
       306, 306, 306, 305, 305, 305, 304, 304, 304, 303, 303, 303, 302,
       302, 302, 301, 301, 301, 300, 300, 300, 299, 299, 299, 298, 298,
       298, 297, 297, 297, 296, 296, 296, 295, 295, 295, 294, 294, 294,
       293, 293, 293, 292, 292, 292, 291, 291, 291, 290, 290, 290, 289,
       289, 289, 288, 288, 288, 287, 287, 287, 286, 286, 286, 285, 285,
       285, 284, 284, 284, 283, 283, 283, 282, 282, 282, 281, 281, 281,
       280, 280, 280, 279, 279, 279, 278, 278, 278, 277, 277, 277, 276,
       276, 276, 275, 275, 275, 274, 274, 274, 273, 273, 273, 27

# Spectral Clustering

In [19]:
from sklearn.cluster import SpectralClustering

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = SpectralClustering(n_clusters=nlable, assign_labels='discretize', 
                         random_state=0).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

Code ran in 0.5849 seconds


SpectralClustering(assign_labels='discretize', n_clusters=56, random_state=0)

In [20]:
res.labels_

array([41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 38, 38,
       38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 34,
       34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
       34, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
       31, 31,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9

# OPTICS

In [21]:
from sklearn.cluster import OPTICS

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = OPTICS(min_samples=2).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

Code ran in 0.4937 seconds


OPTICS(min_samples=2)

In [22]:
res.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Birch

In [23]:
from sklearn.cluster import Birch

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = Birch(n_clusters=nlable).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

Code ran in 0.0551 seconds


Birch(n_clusters=56)

In [24]:
res.labels_

array([39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 49,
       49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 38, 38,
       38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 32, 32, 32, 32, 32, 32,
       32, 32, 32, 32, 32, 32, 32, 32, 32, 32,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3, 36, 36, 36, 36, 36, 36, 36, 36, 36,
       36, 36, 36, 36, 36, 36, 36, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
       31, 31, 31, 31, 31, 31, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
       35, 35, 35, 35, 35, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15,  1,  1